In [1]:
import cv2
import numpy as np
import sys
from PyQt5.QtWidgets import * 
from PyQt5 import uic
from tkinter import *         
from tkinter import filedialog     
import os
import shutil   
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.models import load_model
import tkinter.messagebox
import pyautogui
from scipy.spatial import distance as dt

index_sum = 0
euclidean_sum = 0 
manhattan_sum = 0 
class_A = 0
class_B = 0
class_C = 0
class_D = 0
euclidean_A = 0
euclidean_B = 0
euclidean_C = 0
euclidean_D = 0
manhattan_A = 0
manhattan_B = 0
manhattan_C = 0
manhattan_D = 0

def ask():               
    root = Tk()
    root.title('filedialog study')
 
    root.filename = filedialog.askopenfilename(initialdir='./png',title='파일선택', \
        filetypes=(('mp4 files','*.mp4'),('all files','*.*')))           
 
    root.destroy()

    return root.filename      

def createFolder(directory):   
    try:   
        if not os.path.exists(directory):    
            os.makedirs(directory)       
    except OSError:
        print('Error: Creating directory. ' + directory)
        
def createFolder2(directory):      
    try:   
        if not os.path.exists(directory):    
            os.makedirs(directory)        
        if os.path.exists(directory):    
            shutil.rmtree(directory)    
            os.makedirs(directory)    
    except OSError:
        print('Error: Creating directory. ' + directory)

def image_separation(read_path1, read_path2, read_path3, read_path4, read_path5):
        
    read_path = [read_path1, read_path2, read_path3, read_path4, read_path5]
    
    for p in read_path:
    
        cap = cv2.VideoCapture(p)

        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            lower_red = np.array([0, 50, 80])
            upper_red = np.array([10, 255, 255])
            mask1 = cv2.inRange(hsv, lower_red, upper_red)
            lower_red = np.array([170, 50, 80])
            upper_red = np.array([180, 255, 255])
            mask2 = cv2.inRange(hsv, lower_red, upper_red)
            red_mask = mask1 + mask2
            res = cv2.bitwise_and(frame, frame, mask=red_mask)
        
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    
        mask = cv2.bitwise_or(mask1, mask2)
    
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            cv2.drawContours(res, [cnt], -1, (255, 255, 255), -1)

        src = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        _, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU) 

        cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

        dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
    
        index = 0
        A_class = 0
        B_class = 0
        C_class = 0
        D_class = 0
        A_euclidean = 0
        B_euclidean = 0
        C_euclidean = 0
        D_euclidean = 0
        A_manhattan = 0
        B_manhattan = 0
        C_manhattan = 0
        D_manhattan = 0
    
        for i in range(1, cnt):
            x, y, w, h, area = stats[i]
            euclidean_dist = dt.euclidean([x, y], [x+w, y+h])
            manhattan_dist = dt.cityblock([x, y], [x+w, y+h])
            gray_img = cv2.cvtColor(dst[y:y+h, x:x+w, :], cv2.COLOR_BGR2GRAY)
            thresh = 200
            white_pixels = cv2.threshold(gray_img, thresh, 255, cv2.THRESH_BINARY)[1]
            white_pixel_count = cv2.countNonZero(white_pixels)
            if 50 < area < 150:
                if 10.0 < white_pixel_count < 150.0:
                    index += 1      
                
                    image = dst[y:y+h, x:x+w, :]
                
                    height, width = image.shape[:2]
                    max_dim = max(height, width)
        
                    square_image = np.zeros((max_dim, max_dim, 3), dtype=np.uint8)
                    square_image[:, :] = (0, 0, 0)
        
                    if height > width:
                        offset = (max_dim - width) // 2
                        square_image[:, offset:offset+width] = image
                    else:
                        offset = (max_dim - height) // 2
                        square_image[offset:offset+height, :] = image
        
                    resized_image = cv2.resize(square_image, (50, 50))
                    img_array = img_to_array(resized_image)
                    img_array = np.expand_dims(img_array, axis=0)

                    model = load_model('model.h5', compile = True)

                    pred = model.predict(img_array)
                    class_names = ['A', 'B', 'C', 'D']
                
                    if class_names[np.argmax(pred)] == 'A':
                        A_class += 1
                        A_euclidean += euclidean_dist
                        A_manhattan += manhattan_dist
                    elif class_names[np.argmax(pred)] == 'B':
                        B_class += 1
                        B_euclidean += euclidean_dist
                        B_manhattan += manhattan_dist
                    elif class_names[np.argmax(pred)] == 'C':
                        C_class += 1
                        C_euclidean += euclidean_dist
                        C_manhattan += manhattan_dist
                    elif class_names[np.argmax(pred)] == 'D':
                        D_class += 1
                        D_euclidean += euclidean_dist
                        D_manhattan += manhattan_dist
                
                
            else:
                continue
            
        sum_euclidean = round((A_euclidean + B_euclidean + C_euclidean + D_euclidean)/index, 2)
        sum_manhattan = round((A_manhattan + B_manhattan + C_manhattan + D_manhattan)/index, 2)
    
        if A_class == 0:
            A_euclidean = 0
        else:
            A_euclidean = round(A_euclidean/A_class, 2)
            A_manhattan = round(A_manhattan/A_class, 2)

        if B_class == 0:
            B_euclidean = 0
        else:
            B_euclidean = round(B_euclidean/B_class, 2)
            B_manhattan = round(B_manhattan/B_class, 2)
        
        if C_class == 0:
            C_euclidean = 0
        else:
            C_euclidean = round(C_euclidean/C_class, 2)
            C_manhattan = round(C_manhattan/C_class, 2)
        
        if D_class == 0:
            D_euclidean = 0
        else:
            D_euclidean = round(D_euclidean/D_class, 2)
            D_manhattan = round(D_manhattan/D_class, 2)
            
            
      
        global index_sum 
        global euclidean_sum 
        global manhattan_sum 
        global class_A
        global class_B
        global class_C
        global class_D
        global euclidean_A
        global euclidean_B
        global euclidean_C
        global euclidean_D
        global manhattan_A
        global manhattan_B
        global manhattan_C
        global manhattan_D
        
        index_sum += index
        euclidean_sum += sum_euclidean
        manhattan_sum += sum_manhattan
        class_A += A_class
        class_B += B_class
        class_C += C_class
        class_D += D_class
        euclidean_A += A_euclidean
        euclidean_B += B_euclidean
        euclidean_C += C_euclidean
        euclidean_D += D_euclidean
        manhattan_A += A_manhattan
        manhattan_B += B_manhattan
        manhattan_C += C_manhattan
        manhattan_D += D_manhattan
    
    
    euclidean_sum = round(euclidean_sum/5, 2)
    manhattan_sum = round(manhattan_sum/5, 2)
    
    if class_A == 0:
        euclidean_A = 0
        manhattan_A = 0
    else:
        euclidean_A = round(euclidean_A/5, 2)
        manhattan_A = round(manhattan_A/5, 2)

    if class_B == 0:
        euclidean_B = 0
        manhattan_B = 0
    else:
        euclidean_B = round(euclidean_B/5, 2)
        manhattan_B = round(manhattan_B/5, 2)
        
    if class_C == 0:
        euclidean_C = 0
        manhattan_C = 0
    else:
        euclidean_C = round(euclidean_C/5, 2)
        manhattan_C = round(manhattan_C/5, 2)
        
    if class_D == 0:
        euclidean_D = 0
        manhattan_D = 0
    else:
        euclidean_D = round(euclidean_D/5, 2)
        manhattan_D = round(manhattan_D/5, 2)
        
    
    return index_sum, euclidean_sum, manhattan_sum, class_A, class_B, class_C, class_D, euclidean_A, euclidean_B, \
euclidean_C, euclidean_D, manhattan_A, manhattan_B, manhattan_C, manhattan_D
    
    
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.ui = uic.loadUi("GUI_BASE.ui", self) 
        self.push.clicked.connect(self.ask)          
        self.push2.clicked.connect(self.result)
        self.show() 
    
    def gotoScreen1(self):
        self.hide()
        window_2 = SubWindow1()
        window_2.exec_()
        
    def ask(self):
        read_path = ask()
        if self.tableWidget.item(0, 2).text() == "":
            self.tableWidget.setItem(0, 0, QTableWidgetItem('1'))
            self.tableWidget.setItem(0, 1, QTableWidgetItem('mp4'))
            self.tableWidget.setItem(0, 2, QTableWidgetItem(read_path))
        elif self.tableWidget.item(0, 2).text() != "":
            if self.tableWidget.item(1, 2).text() == "":
                self.tableWidget.setItem(1, 0, QTableWidgetItem('2'))
                self.tableWidget.setItem(1, 1, QTableWidgetItem('mp4'))
                self.tableWidget.setItem(1, 2, QTableWidgetItem(read_path))
            elif self.tableWidget.item(1, 2).text() != "":
                if self.tableWidget.item(2, 2).text() == "":
                    self.tableWidget.setItem(2, 0, QTableWidgetItem('3'))
                    self.tableWidget.setItem(2, 1, QTableWidgetItem('mp4'))
                    self.tableWidget.setItem(2, 2, QTableWidgetItem(read_path))
                elif self.tableWidget.item(2, 2).text() != "":
                    if self.tableWidget.item(3, 2).text() == "":
                        self.tableWidget.setItem(3, 0, QTableWidgetItem('4'))
                        self.tableWidget.setItem(3, 1, QTableWidgetItem('mp4'))
                        self.tableWidget.setItem(3, 2, QTableWidgetItem(read_path))
                    elif self.tableWidget.item(3, 2).text() != "":
                        self.tableWidget.setItem(4, 0, QTableWidgetItem('5'))
                        self.tableWidget.setItem(4, 1, QTableWidgetItem('mp4'))
                        self.tableWidget.setItem(4, 2, QTableWidgetItem(read_path))


    def result(self):
        text1 = self.tableWidget.item(0, 2).text()
        text2 = self.tableWidget.item(1, 2).text()
        text3 = self.tableWidget.item(2, 2).text()
        text4 = self.tableWidget.item(3, 2).text()
        text5 = self.tableWidget.item(4, 2).text()
        if not os.path.isfile(text5):
            pyautogui.alert('CASA 영상 5개를 업로드 해주세요.')
        if os.path.isfile(text5):
            self.hide()
            path = SubWindow1(text1, text2, text3, text4, text5)
            path.exec_()
            

            
            
class SubWindow1(QDialog):
    def __init__(self, text1, text2, text3, text4, text5):
        super().__init__()
        self.ui = uic.loadUi("GUI_BASE2.ui", self) 
        self.show()
        self.label_32.setText("분석 중입니다...")
        
        index_sum, euclidean_sum, manhattan_sum, class_A, class_B, class_C, class_D, euclidean_A, euclidean_B, \
        euclidean_C, euclidean_D, manhattan_A, manhattan_B, manhattan_C, manhattan_D  = image_separation(text1, text2, text3, text4, text5)
            
        self.label.setText("평균 속도        " + str(euclidean_sum))
        self.label_2.setText("평균 유클리드        " + str(euclidean_sum))
        self.label_16.setText("평균 맨하탄        " + str(manhattan_sum))
        self.label_3.setText("평균 개수        " + str(index_sum))
        self.label_4.setText("속도        " + str(euclidean_A))
        self.label_5.setText("유클리드        " + str(euclidean_A))
        self.label_17.setText("맨하탄        " + str(manhattan_A))
        self.label_6.setText("개수        " + str(class_A))
        self.label_7.setText("속도        " + str(euclidean_B))
        self.label_8.setText("유클리드        " + str(euclidean_B))
        self.label_18.setText("맨하탄        " + str(manhattan_B))
        self.label_9.setText("개수        " + str(class_B))
        self.label_10.setText("속도        " + str(euclidean_C))
        self.label_11.setText("유클리드        " + str(euclidean_C))
        self.label_19.setText("맨하탄        " + str(manhattan_C))
        self.label_12.setText("개수        " + str(class_C))
        self.label_13.setText("속도        " + str(euclidean_D))
        self.label_14.setText("유클리드        " + str(euclidean_D))
        self.label_20.setText("맨하탄        " + str(manhattan_D))
        self.label_15.setText("개수        " + str(class_D))
        self.label_32.setText("분석 결과가 도착했습니다.")
        
if __name__ == "__main__":     
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    app.exec_()

1/1 [==============================] - 0s 38ms/step
